# Imports

In [ ]:
do_ZeroMean_simulation = True
do_FHMean_simulation = True
do_PhysicsElsewhere_simulation = True

In [ ]:
import json
from pathlib import Path

PROJECT_ROOT = Path.cwd().parent

CONFIG_PATH = PROJECT_ROOT / "param_config.json"
with open(CONFIG_PATH, "r") as f:
    param_config = json.load(f)

In [ ]:
import sys, os
sys.path.append(os.path.abspath(".."))

import source_files.ExperimentalFunctions as EFF
import source_files.UtilityFunctions as UF
import source_files.ConvergenceTesting as CT
import source_files.InitializationFunctions as IF
import source_files.PhysicsElsewhereFunctions as PEF

In [ ]:
UF.set_FH_prior_config(param_config)

In [ ]:
import matplotlib.pyplot as plt
from gpcam.gp_optimizer import GPOptimizer
from tqdm import tqdm
import time
import numpy as np
import pandas as pd

# Simulation Comparison of Different Methods

## General Parameters Shared Across Workflows

In [ ]:
def hp_has_converged_bounds_scaled_linear(
    hp_log,
    hp_bounds,
    window,
    tol,
):
    """Check hyperparameter convergence using boundary-scaled step sizes compared to the specified tolerance"""
    if len(hp_log) < window + 1:
        return False

    H = np.asarray(hp_log[-(window + 1):], dtype=float)
    B = np.asarray(hp_bounds, dtype=float)

    dH = np.abs(np.diff(H, axis=0))
    ranges = B[:, 1] - B[:, 0]

    # numerical safety
    ranges = np.clip(ranges, 1e-12, None)

    norm_steps = dH / ranges

    return np.max(norm_steps) < tol


In [ ]:
def build_hyperparameters(cfg, model_name):
    """Using the parameter configuration file, build the corresponding initial HP list and HP boundary list based on the specified model name"""
    
    hp_all = cfg["general_parameters"]["hyperparameters"]
    model_cfg = cfg["simulation_parameters"]["models"][model_name]

    init_HPs = [hp_all[k]["init"] for k in model_cfg["active_hyperparameters"]]
    HP_bounds = [hp_all[k]["bounds"] for k in model_cfg["active_hyperparameters"]]

    if model_name == "Physics-Elsewhere":
        init_HPs.append(cfg["simulation_parameters"]["models"][model_name]["additional_hyperparameter"]["lchi"]["init"])
        HP_bounds.append(cfg["simulation_parameters"]["models"][model_name]["additional_hyperparameter"]["lchi"]["bounds"])

    return init_HPs, HP_bounds

In [ ]:
np_RNG_seed = param_config["general_parameters"]["np_RNG_seed"]

N_campaigns = param_config["simulation_parameters"]["N_campaigns"] #number of trials per workflow

parameter_bounds = [param_config["general_parameters"]["parameter_bounds"]["composition"],
                    param_config["general_parameters"]["parameter_bounds"]["temperature_C"]]

a_value_noise = param_config["general_parameters"]["a_value_noise"] 
floor_noise = param_config["general_parameters"]["floor_noise"]

max_iters = param_config["simulation_parameters"]["max_iterations"]

n_points_per_iteration = param_config["general_parameters"]["n_points_per_iteration"]

N_window_HPs = param_config["simulation_parameters"]["N_window_HPs"]
tol_HP_conv = param_config["simulation_parameters"]["tol_HP_convergence"]

rounding_precision_composition = param_config["general_parameters"]["rounding_precision"]["composition"]
rounding_precision_temperature = param_config["general_parameters"]["rounding_precision"]["temperature_C"]

In [ ]:
#use same list of 4 points, measurements, and variances to seed each of the runs of all types of simulated processes
initial_xT_values = param_config["simulation_parameters"]["initial_xT_values"]

initial_measurement_values = [UF.ground_truth(xT[0], xT[1]) for xT in initial_xT_values]

initial_variance_values = [a_value_noise * (meas**2) for meas in initial_measurement_values]

In [ ]:
def zero_prior_mean(x, hyperparams):
    """Zero-value GP prior mean""" 
    N_points = x.shape[0]
    prior_mean_ndarray = np.zeros((N_points,), dtype=float)

    return prior_mean_ndarray

In [ ]:
def initialize_GP_instance(init_xT_vals, init_meas_vals, init_vars_vals, init_HPs, prior_type,):
    """Initializes a gpCAM GP instance with the specified prior mean and kernel forms based on the input 'prior_type'"""
    
    if prior_type == "FH":
        my_GP_opt = GPOptimizer(x_data = np.array(init_xT_vals),
                                y_data = np.array(init_meas_vals),
                                init_hyperparameters = np.array(init_HPs),
                                noise_variances = np.array(init_vars_vals),
                                gp_kernel_function = IF.custom_kernel,
                                gp_mean_function = UF.flory_huggins_prior_mean,)
        return my_GP_opt
        
    elif prior_type == "ZM": 
        my_GP_opt = GPOptimizer(x_data = np.array(init_xT_vals),
                                y_data = np.array(init_meas_vals),
                                init_hyperparameters = np.array(init_HPs),
                                noise_variances = np.array(init_vars_vals),
                                gp_kernel_function = IF.custom_kernel,
                                gp_mean_function = zero_prior_mean)
        return my_GP_opt

    elif prior_type == "ZM_FHKernel":
        my_GP_opt = GPOptimizer(x_data = np.array(init_xT_vals),
                                y_data = np.array(init_meas_vals),
                                init_hyperparameters = np.array(init_HPs),
                                noise_variances = np.array(init_vars_vals),
                                gp_kernel_function = PEF.FH_kernel,
                                gp_mean_function = zero_prior_mean)

        return my_GP_opt

    else:
        raise ValueError(f"Unsupported Prior Type: {prior_type}")

## Code Execution - Zero Prior Mean

In [ ]:
if isinstance(np_RNG_seed, int):
    np.random.seed(np_RNG_seed) #seed implemented as np_RNG_seed % 2^32 (4294967296)

In [ ]:
ZM_outer_bound_conv_log = []
ZM_outer_HP_log = []
ZM_outer_converged_idxs = []
ZM_outer_GP_opt_log = []

ZM_init_HPs, ZM_HP_bounds = build_hyperparameters(param_config, "Zero-mean")

if do_ZeroMean_simulation == True:
    for ZM_campaign_ind in range(N_campaigns):
        print('=='*20)
        print(f"Campaign {ZM_campaign_ind + 1} / {N_campaigns}")
        print('=='*20)
    
        ZM_ind_bound_conv_log = []
        ZM_ind_HP_log = []
    
        curr_ZM_xT_values = [init_xT.copy() for init_xT in initial_xT_values]
        curr_ZM_measurement_values = initial_measurement_values.copy()
        curr_ZM_variance_values = initial_variance_values.copy()
        
        ind_ZM_GP_opt = initialize_GP_instance(curr_ZM_xT_values, 
                                               curr_ZM_measurement_values, 
                                               curr_ZM_variance_values, 
                                               ZM_init_HPs, "ZM")
        
        #initial training using the 4 initial points
        curr_ZM_HPs = ind_ZM_GP_opt.train(hyperparameter_bounds = np.array(ZM_HP_bounds), 
                                          init_hyperparameters = ind_ZM_GP_opt.hyperparameters, 
                                          method = "global",)
        ZM_ind_HP_log.append(curr_ZM_HPs)
        
        curr_ZM_tqdm_bar = tqdm(range(max_iters))

        for curr_ZM_iteration in curr_ZM_tqdm_bar:

            curr_ZM_next_points, _, _, _ = UF.get_next_point_s(ind_ZM_GP_opt, 
                                                               prec_comp = rounding_precision_composition, 
                                                               prec_temp = rounding_precision_temperature, 
                                                               parameter_bounds = parameter_bounds,
                                                               n_points_per_stack = n_points_per_iteration,)
            
            curr_ZM_next_points=np.array(curr_ZM_next_points)
            curr_ZM_next_meas = UF.ground_truth(curr_ZM_next_points[:,0], 
                                                curr_ZM_next_points[:,1])
            curr_ZM_next_vars = np.array([a_value_noise * (meas**2) for meas in curr_ZM_next_meas])
        
            ind_ZM_GP_opt.tell(curr_ZM_next_points, 
                               curr_ZM_next_meas, 
                               curr_ZM_next_vars, 
                               append = True)
            
            ind_ZM_GP_opt.train(hyperparameter_bounds = np.array(ZM_HP_bounds), 
                                init_hyperparameters = ind_ZM_GP_opt.hyperparameters, 
                                method = "global",)

            
            ZM_ind_HP_log.append(ind_ZM_GP_opt.hyperparameters)

            ZM_ind_curr_converged_bool_HPs = hp_has_converged_bounds_scaled_linear(ZM_ind_HP_log,
                                                                                   np.array(ZM_HP_bounds),
                                                                                   window = N_window_HPs,
                                                                                   tol = tol_HP_conv)
                                                                                   
            if ZM_ind_curr_converged_bool_HPs == True:
                ZM_outer_converged_idxs.append(curr_ZM_iteration+1) 
                
                ZM_outer_GP_opt_log.append(initialize_GP_instance(ind_ZM_GP_opt.x_data, 
                                                                  ind_ZM_GP_opt.y_data, 
                                                                  ind_ZM_GP_opt.likelihood.V, 
                                                                  ind_ZM_GP_opt.hyperparameters, 
                                                                  "ZM",))
                ZM_outer_HP_log.append(ZM_ind_HP_log)

                break
    
    
    
    
    

## Code Execution - Flory-Huggins Prior

In [ ]:
if isinstance(np_RNG_seed, int):
    np.random.seed(np_RNG_seed) #seed implemented as np_RNG_seed % 2^32 (4294967296)

In [ ]:
FH_outer_bound_conv_log = []
FH_outer_HP_log = []
FH_outer_converged_idxs = []
FH_outer_GP_opt_log = []

FH_init_HPs, FH_HP_bounds = build_hyperparameters(param_config, "Flory-Huggins")

if do_FHMean_simulation == True:
    for FH_campaign_ind in range(N_campaigns):
        print('=='*20)
        print(f"Campaign {FH_campaign_ind + 1} / {N_campaigns}")
        print('=='*20)
    
        FH_ind_bound_conv_log = []
        FH_ind_HP_log = []
    
        curr_FH_xT_values = [init_xT.copy() for init_xT in initial_xT_values]
        curr_FH_measurement_values = initial_measurement_values.copy()
        curr_FH_variance_values = initial_variance_values.copy()
        
        ind_FH_GP_opt = initialize_GP_instance(curr_FH_xT_values, 
                                               curr_FH_measurement_values, 
                                               curr_FH_variance_values, 
                                               FH_init_HPs, "FH")
        
        #initial training using the 4 initial points
        curr_FH_HPs = ind_FH_GP_opt.train(hyperparameter_bounds = np.array(FH_HP_bounds), 
                                          init_hyperparameters = ind_FH_GP_opt.hyperparameters, 
                                          method = "global",)
        FH_ind_HP_log.append(curr_FH_HPs)
        
        curr_FH_tqdm_bar = tqdm(range(max_iters))
    
        for curr_FH_iteration in curr_FH_tqdm_bar:

            curr_FH_next_points, _, _, _ = UF.get_next_point_s(ind_FH_GP_opt, 
                                                               prec_comp = rounding_precision_composition, 
                                                               prec_temp = rounding_precision_temperature, 
                                                               parameter_bounds = parameter_bounds,
                                                               n_points_per_stack = n_points_per_iteration,)
            
            
            curr_FH_next_points=np.array(curr_FH_next_points)
            curr_FH_next_meas = UF.ground_truth(curr_FH_next_points[:,0], 
                                                curr_FH_next_points[:,1])
            curr_FH_next_vars = np.array([a_value_noise * (meas**2) for meas in curr_FH_next_meas])
        
            ind_FH_GP_opt.tell(curr_FH_next_points, 
                               curr_FH_next_meas, 
                               curr_FH_next_vars, 
                               append = True)

            ind_FH_GP_opt.train(hyperparameter_bounds = np.array(FH_HP_bounds), 
                                init_hyperparameters = ind_FH_GP_opt.hyperparameters, 
                                method = "global",)
            
            FH_ind_HP_log.append(ind_FH_GP_opt.hyperparameters)


            FH_ind_curr_converged_bool_HPs = hp_has_converged_bounds_scaled_linear(FH_ind_HP_log,
                                                                                  np.array(FH_HP_bounds),
                                                                                   window = N_window_HPs,
                                                                                   tol = tol_HP_conv)

            
            if FH_ind_curr_converged_bool_HPs == True:
                FH_outer_converged_idxs.append(curr_FH_iteration+1) 
                
                FH_outer_GP_opt_log.append(initialize_GP_instance(ind_FH_GP_opt.x_data, 
                                                                  ind_FH_GP_opt.y_data, 
                                                                  ind_FH_GP_opt.likelihood.V, 
                                                                  ind_FH_GP_opt.hyperparameters, 
                                                                  "FH",))
    
                FH_outer_HP_log.append(FH_ind_HP_log)

                
                break

## Code Execution - "Physics Elsewhere"

In [ ]:
if isinstance(np_RNG_seed, int):
    np.random.seed(np_RNG_seed) #seed implemented as np_RNG_seed % 2^32 (4294967296)

In [ ]:
PE_outer_bound_conv_log = []
PE_outer_HP_log = []
PE_outer_converged_idxs = []
PE_outer_GP_opt_log = []

PE_init_HPs, PE_HP_bounds = build_hyperparameters(param_config, "Physics-Elsewhere")

if do_PhysicsElsewhere_simulation == True:
    #HP Initialization


    for PE_campaign_ind in range(N_campaigns):
        print('=='*20)
        print(f"Campaign {PE_campaign_ind + 1}/{N_campaigns}")
        print('=='*20)

        PE_ind_bound_conv_log = []
        PE_ind_HP_log = []
    
        curr_PE_xT_values = [init_xT.copy() for init_xT in initial_xT_values]
        curr_PE_measurement_values = initial_measurement_values.copy()
        curr_PE_variance_values = initial_variance_values.copy()
        
        ind_PE_GP_opt = initialize_GP_instance(curr_PE_xT_values, 
                                               curr_PE_measurement_values, 
                                               curr_PE_variance_values, 
                                               PE_init_HPs, "ZM_FHKernel")

        curr_PE_HPs = ind_PE_GP_opt.train(hyperparameter_bounds = np.array(PE_HP_bounds), 
                                          init_hyperparameters = ind_PE_GP_opt.hyperparameters, 
                                          method = "global",)
        PE_ind_HP_log.append(curr_PE_HPs)
        
        curr_PE_tqdm_bar = tqdm(range(max_iters))

        for curr_PE_iteration in curr_PE_tqdm_bar:

            curr_PE_next_points, _, _, _ = UF.get_next_point_s(ind_PE_GP_opt, 
                                                               prec_comp = rounding_precision_composition, 
                                                               prec_temp = rounding_precision_temperature, 
                                                               parameter_bounds = parameter_bounds,
                                                               n_points_per_stack = n_points_per_iteration,)
            

            
            curr_PE_next_points=np.array(curr_PE_next_points)
            curr_PE_next_meas = UF.ground_truth(curr_PE_next_points[:,0], 
                                                curr_PE_next_points[:,1])
            curr_PE_next_vars = np.array([a_value_noise * (meas**2) for meas in curr_PE_next_meas])
        
            ind_PE_GP_opt.tell(curr_PE_next_points, curr_PE_next_meas, curr_PE_next_vars, append = True)

            ind_PE_GP_opt.train(hyperparameter_bounds = np.array(PE_HP_bounds), 
                                init_hyperparameters = ind_PE_GP_opt.hyperparameters, 
                                method = "global",)
            
            
            PE_ind_HP_log.append(ind_PE_GP_opt.hyperparameters)
            
            PE_ind_curr_converged_bool_HPs = hp_has_converged_bounds_scaled_linear(PE_ind_HP_log,
                                                                                  np.array(PE_HP_bounds),
                                                                                  window = N_window_HPs,
                                                                                   tol = tol_HP_conv)
            
            if PE_ind_curr_converged_bool_HPs == True:
                PE_outer_converged_idxs.append(curr_PE_iteration+1) 
                
                PE_outer_GP_opt_log.append(initialize_GP_instance(ind_PE_GP_opt.x_data, 
                                                                  ind_PE_GP_opt.y_data, 
                                                                  ind_PE_GP_opt.likelihood.V, 
                                                                  ind_PE_GP_opt.hyperparameters, 
                                                                  "ZM_FHKernel",))
                PE_outer_HP_log.append(PE_ind_HP_log)

                
                break

# Convergence Analysis - Hyperparameters

In [ ]:
hp_names_FH = ["$\sigma_f^2$", "$l_x$ (wt. frac. PMMA)","$l_T$ (°C)",'A','B (K)']
hp_names_ZM = hp_names_FH[:3]
hp_names_PE = hp_names_ZM + ["$l_\chi$"]

ylim_list_FH =  [ [1e-2, 1e1], [1e-3, 1e0], [1e-1, 1e2], [-1, 1], [-200, 200] ]
ylim_list_ZM = ylim_list_FH[:3]

In [ ]:
if do_ZeroMean_simulation == True:

    ZM_HP_aspect_ratio = 2.5
    ZM_HP_scale = 4
    
    ZM_HP_fig, ZM_HP_axes = plt.subplots(1,3, figsize = (ZM_HP_aspect_ratio * ZM_HP_scale, ZM_HP_scale))
    ZM_HP_axes = ZM_HP_axes.flatten()
    
    for ZM_HP_ind in range(3):
        curr_ZM_HP_ax = ZM_HP_axes[ZM_HP_ind]
    
        for campaign_ZM_HP_log in ZM_outer_HP_log:
            curr_ZM_HP_arr = np.array(campaign_ZM_HP_log)   
            curr_ZM_HP_ax.plot(curr_ZM_HP_arr[:, ZM_HP_ind], alpha=1)
    
        curr_ZM_HP_ax.set_title(hp_names_ZM[ZM_HP_ind])
        curr_ZM_HP_ax.set_xlabel("Iteration")
        # curr_ZM_HP_ax.set_ylabel("Value")
        curr_ZM_HP_ax.set_ylim(ZM_HP_bounds[ZM_HP_ind])
    
    plt.suptitle("Zero Mean HP Monitoring")
    plt.tight_layout()
    plt.show()

In [ ]:
if do_FHMean_simulation == True:
    FH_HP_aspect_ratio = 1.5
    FH_HP_scale = 6
    
    FH_HP_fig, FH_HP_axes = plt.subplots(2,3, figsize = (FH_HP_aspect_ratio * FH_HP_scale, FH_HP_scale))
    
    FH_HP_axes = FH_HP_axes.flatten()
    
    for FH_HP_ind in range(5):
        curr_FH_HP_ax = FH_HP_axes[FH_HP_ind]
    
        for campaign_FH_HP_log in FH_outer_HP_log:
            curr_FH_HP_arr = np.array(campaign_FH_HP_log)   
            curr_FH_HP_ax.plot(curr_FH_HP_arr[:, FH_HP_ind], alpha=1)
    
        curr_FH_HP_ax.set_title(hp_names_FH[FH_HP_ind])
        curr_FH_HP_ax.set_xlabel("Iteration")
        # curr_FH_HP_ax.set_ylabel("Value")
        curr_FH_HP_ax.set_ylim(FH_HP_bounds[FH_HP_ind])
    
    FH_HP_fig.delaxes(FH_HP_axes[-1])
    
    plt.suptitle("Flory-Huggins HP Monitoring")
    plt.tight_layout()
    plt.show()

In [ ]:
if do_PhysicsElsewhere_simulation == True:
    PE_HP_aspect_ratio = 1.5
    PE_HP_scale = 8

    PE_HP_fig, PE_HP_axes = plt.subplots(2,2, figsize = (PE_HP_aspect_ratio * PE_HP_scale, PE_HP_scale))
    PE_HP_axes = PE_HP_axes.flatten()

    for PE_HP_ind in range(4):
        curr_PE_HP_ax = PE_HP_axes[PE_HP_ind]
        for campaign_PE_HP_log in PE_outer_HP_log:
            curr_PE_HP_arr = np.array(campaign_PE_HP_log)   
            curr_PE_HP_ax.plot(curr_PE_HP_arr[:, PE_HP_ind], alpha=1)
    
        curr_PE_HP_ax.set_title(hp_names_PE[PE_HP_ind])
        curr_PE_HP_ax.set_xlabel("Iteration")
        # curr_FH_HP_ax.set_ylabel("Value")
        curr_PE_HP_ax.set_ylim(PE_HP_bounds[PE_HP_ind])

    plt.suptitle("Physics-Elsewhere (Kernel) HP Monitoring")
    plt.tight_layout()
    plt.show()

# Convergence Analysis - Iterations to Convergence

In [ ]:
iter_to_conv_summary = pd.DataFrame({
    "Mean": [np.mean(ZM_outer_converged_idxs),
            np.mean(FH_outer_converged_idxs),
            np.mean(PE_outer_converged_idxs)],
    "Std Dev":[f"{np.std(ZM_outer_converged_idxs):.2f}",
              f"{np.std(FH_outer_converged_idxs):.2f}",
              f"{np.std(PE_outer_converged_idxs):.2f}"]
}, index = ["Zero-prior mean", "Flory-Huggins-informed prior", "Flory-Huggins-augmented kernel"])

In [ ]:
print("Iterations Required for Hyperparameter Convergence")
print("--"*20)
print(iter_to_conv_summary)